In [1]:
from google.colab import drive
isMount=!df |grep /content/drive
if len(isMount)<1: drive.mount('/content/drive')


Mounted at /content/drive


# CNN Example From Flowers by Keras (Color Version)


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras import datasets, layers, models, regularizers

import numpy as np
import matplotlib.pyplot as plt



In [3]:
from os import listdir
from os.path import isfile, join
from pylab import *
from numpy import *

def getFolder(thePath,isFile=True):
    return [f for f in listdir(thePath) if isFile == isfile(join(thePath, f)) ]


def getImagesAndLabels(tPath):
    labels=getFolder(tPath,False)
    encoder={w: i for i, w in enumerate(labels)}
    decoder={i: w  for i, w in enumerate(labels)}

    
    tImages,tLabels=None,None

    for label in labels:
        imgFolder=join(tPath,label)
        files= [f for f in listdir(imgFolder) if isfile(join(imgFolder, f))]
        for file in files:
            imageFile=join(imgFolder,file)
            img=np.array([imread(imageFile)])/255.
            if tImages is None:
                tImages, tLabels =img, np.array([encoder[label]],ndmin=1)
            else:
#                 print(tImages.shape,img.shape,imageFile)
                tImages = np.vstack( (tImages,img) )
                tLabels = np.append(tLabels,np.array([encoder[label]]) ,axis=0) 
        print(label,tImages.shape)                 
    return (tImages,tLabels,encoder,decoder)

    

In [4]:
%%bash
x=$(dpkg -l|grep imagemagick);[ ${#x} -eq 0 ]&&sudo apt install -y imagemagick
[ ! -f /content/drive/MyDrive/Lec_Capture/_data/flower_photos.tgz  ]&&wget http://download.tensorflow.org/example_images/flower_photos.tgz -O /content/drive/MyDrive/Lec_Capture/_data/flower_photos.tgz 
tar xzf /content/drive/MyDrive/Lec_Capture/_data/flower_photos.tgz 
find ./flower_photos|grep jpg|xargs -i -t convert {} -quiet -resize 300x224\! -quality 100 -colorspace RGB -type truecolor {} 2>> /tmp/log
find ./flower_photos|grep jpg|head -n 10|xargs -i -t identify {} 2>>/tmp/log


Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono ghostscript gsfonts
  imagemagick-6-common imagemagick-6.q16 libcupsfilters1 libcupsimage2
  libdjvulibre-text libdjvulibre21 libgs9 libgs9-common libijs-0.35
  libjbig2dec0 liblqr-1-0 libmagickcore-6.q16-3 libmagickcore-6.q16-3-extra
  libmagickwand-6.q16-3 libnetpbm10 libwmf0.2-7 netpbm poppler-data
Suggested packages:
  fonts-noto ghostscript-x imagemagick-doc autotrace cups-bsd | lpr | lprng
  enscript gimp gnuplot grads hp2xx html2ps libwmf-bin mplayer povray radiance
  sane-utils texlive-base-bin transfig ufraw-batch inkscape libjxr-tools
  libwmf0.2-7-gtk poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono ghostscript gsf



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 23.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [ ]:
from sklearn.model_selection import train_test_split

basePath='./flower_photos'
imgs,labels,encoder,decoder=getImagesAndLabels(basePath)

class_cnt=len(encoder.keys())

train_images,test_images,train_labels,test_labels=train_test_split(imgs,labels,test_size=0.3333, random_state=99)

roses (641, 224, 300, 3)
dandelion (1539, 224, 300, 3)
daisy (2172, 224, 300, 3)
tulips (2971, 224, 300, 3)


In [ ]:
imgShape=train_images[0].shape
img_width,img_hight,color_count=imgShape
train_images = train_images.reshape((-1,img_width,img_hight,color_count ))
test_images = test_images.reshape((-1, img_width,img_hight,color_count ))

print("Shape of Train_images = ",train_images.shape)

In [ ]:

model = models.Sequential()
model.add(layers.Conv2D(128, (11, 11), activation='relu',padding='same',kernel_regularizer=regularizers.l2(0.02), input_shape=imgShape))
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(64, (7, 7), activation='relu',padding='same',kernel_regularizer=regularizers.l2(0.02)))
model.add(layers.Dropout(0.2))
model.add(layers.MaxPooling2D((4, 4)))

model.add(layers.Conv2D(4, (3, 3), activation='relu',padding='same',kernel_regularizer=regularizers.l2(0.02)))
model.add(layers.Dropout(0.2))
model.add(layers.MaxPooling2D((2, 2)))


model.add(layers.Flatten())
model.add(layers.Dense(64, activation='sigmoid',kernel_regularizer=regularizers.l2(0.02)))
model.add(layers.Dense(class_cnt, activation='softmax'))
# model.summary()


In [ ]:
from tensorflow.python.keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.0002),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=50,batch_size=10)

In [ ]:
score = model.evaluate(test_images, test_labels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# Reularization 구현해 보세요.